In [1]:
import pandas as pd
df_test = pd.read_hdf('../data/processed/test/aggr_test.hdf')
df_test.head()

,ID,shop_id,item_id,year,month,item_name,item_category_id,item_category_name,big_category,date,...,count_aggr_6_months_shop_id_item_category_id,count_aggr_2_quartals_ago_shop_id_item_category_id,count_aggr_year_shop_id_item_category_id,count_aggr_1_month_shop_id_item_category_id,count_aggr_3_months_shop_id_big_category,count_aggr_1_month_ago_shop_id_big_category,count_aggr_6_months_shop_id_big_category,count_aggr_2_quartals_ago_shop_id_big_category,count_aggr_year_shop_id_big_category,count_aggr_1_month_shop_id_big_category
0,0,5,5037,2015,11,"NHL 15 [PS3, русские субтитры]",19,Игры - PS3,Игры,2015-11-01,...,531.0,181.0,1056.0,75.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5100,4,5037,2015,11,"NHL 15 [PS3, русские субтитры]",19,Игры - PS3,Игры,2015-11-01,...,309.0,134.0,689.0,34.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10200,6,5037,2015,11,"NHL 15 [PS3, русские субтитры]",19,Игры - PS3,Игры,2015-11-01,...,403.0,178.0,1041.0,54.0,0.0,0.0,0.0,0.0,0.0,0.0
3,15300,3,5037,2015,11,"NHL 15 [PS3, русские субтитры]",19,Игры - PS3,Игры,2015-11-01,...,355.0,166.0,853.0,45.0,0.0,0.0,0.0,0.0,0.0,0.0
4,20400,2,5037,2015,11,"NHL 15 [PS3, русские субтитры]",19,Игры - PS3,Игры,2015-11-01,...,546.0,197.0,1286.0,70.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
df_test['item_cnt_month'] = df_test['count_aggr_1_month_ago_shop_id_item_id']
df_test['item_cnt_month'].isnull().sum()

0

In [4]:
df_test['item_cnt_month'] = df_test['item_cnt_month'].clip(0, 20)
df_test[['ID', 'item_cnt_month']].to_csv('../data/submissions/prev_value_benchmark.csv', index=False)

In [6]:
total = pd.read_hdf('../data/processed/train/total.hdf')

In [10]:
cnts = total[(total['month'] == 10) & (total['year'] == 2015)].groupby(['shop_id', 'item_id'], as_index=False)['item_cnt_day'].sum()
cnts.head()

,shop_id,item_id,item_cnt_day
0,2,31,1.0
1,2,486,3.0
2,2,787,1.0
3,2,794,1.0
4,2,968,1.0


In [14]:
df_test_manually_made = df_test[['ID', 'shop_id', 'item_id']].copy()
df_test_manually_made = pd.merge(
    df_test_manually_made, cnts,
    on=['shop_id', 'item_id'],
    how='left'
).fillna(0)
df_test_manually_made.rename(columns={'item_cnt_day': 'item_cnt_month'}, inplace=True)
df_test_manually_made['item_cnt_month'].clip(0, 20, inplace=True)

In [25]:
df_test.columns

Index(['ID', 'shop_id', 'item_id', 'year', 'month', 'item_name',
       'item_category_id', 'item_category_name', 'big_category', 'date',
       'aggr_from_date', 'aggr_to_date', 'count_aggr_3_months_shop_id',
       'count_aggr_1_month_ago_shop_id', 'count_aggr_6_months_shop_id',
       'count_aggr_2_quartals_ago_shop_id', 'count_aggr_year_shop_id',
       'count_aggr_1_month_shop_id', 'count_aggr_3_months_item_id',
       'count_aggr_1_month_ago_item_id', 'count_aggr_6_months_item_id',
       'count_aggr_2_quartals_ago_item_id', 'count_aggr_year_item_id',
       'count_aggr_1_month_item_id', 'count_aggr_3_months_item_category_id',
       'count_aggr_1_month_ago_item_category_id',
       'count_aggr_6_months_item_category_id',
       'count_aggr_2_quartals_ago_item_category_id',
       'count_aggr_year_item_category_id',
       'count_aggr_1_month_item_category_id',
       'count_aggr_3_months_big_category',
       'count_aggr_1_month_ago_big_category',
       'count_aggr_6_months_big

In [37]:
df_wrong_rows = df_test[df_test_manually_made['item_cnt_month'] != df_test['count_aggr_1_month_shop_id_item_id']][
    ['count_aggr_1_month_ago_shop_id_item_id', 'count_aggr_1_month_shop_id_item_id']
]
df_wrong_rows['manually_made'] = df_test_manually_made[df_test_manually_made['item_cnt_month'] != df_test['count_aggr_1_month_shop_id_item_id']]['item_cnt_month']
df_wrong_rows

,count_aggr_1_month_ago_shop_id_item_id,count_aggr_1_month_shop_id_item_id,manually_made
7,-1.0,1.0,0.0
261,49.0,26.0,20.0
263,36.0,22.0,20.0
2840,-1.0,1.0,0.0
5719,-1.0,1.0,0.0
8789,0.0,21.0,20.0
11015,0.0,24.0,20.0
27890,-1.0,1.0,0.0
34322,31.0,21.0,20.0
34323,45.0,32.0,20.0


In [41]:
df_test.loc[7]['count_aggr_1_month_shop_id_item_id']

1.0

In [46]:
total[
    (total['shop_id'] == 12) &
    (total['item_id'] == 5037) &
    (total['year'] == 2015) &
    (total['month'] == 10)
]

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,day,month,year,weekday,...,to_new_year_days,is_feb_special_time,feb_special_gaussian,from_feb_special_days,to_feb_special_days,is_nov_special_time,nov_special_gaussian,from_nov_special_days,to_nov_special_days,is_holiday


In [42]:
df_test.loc[7]

ID                                                                             35700
shop_id                                                                           12
item_id                                                                         5037
year                                                                            2015
month                                                                             11
item_name                                             NHL 15 [PS3, русские субтитры]
item_category_id                                                                  19
item_category_name                                                        Игры - PS3
big_category                                                                    Игры
date                                                             2015-11-01 00:00:00
aggr_from_date                                                   2015-10-01 00:00:00
aggr_to_date                                                     

In [36]:
df_wrong_rows['manually_made'].value_counts()

20.0    185
0.0      22
Name: manually_made, dtype: int64

In [33]:
df_test[]

,ID,shop_id,item_id,year,month,item_name,item_category_id,item_category_name,big_category,date,...,count_aggr_2_quartals_ago_shop_id_item_category_id,count_aggr_year_shop_id_item_category_id,count_aggr_1_month_shop_id_item_category_id,count_aggr_3_months_shop_id_big_category,count_aggr_1_month_ago_shop_id_big_category,count_aggr_6_months_shop_id_big_category,count_aggr_2_quartals_ago_shop_id_big_category,count_aggr_year_shop_id_big_category,count_aggr_1_month_shop_id_big_category,item_cnt_month
7,35700,12,5037,2015,11,"NHL 15 [PS3, русские субтитры]",19,Игры - PS3,Игры,2015-11-01,...,132.0,877.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
261,45958,31,3350,2015,11,"FIFA 16 [PS3, русская версия]",19,Игры - PS3,Игры,2015-11-01,...,600.0,3405.0,174.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0
263,56158,25,3350,2015,11,"FIFA 16 [PS3, русская версия]",19,Игры - PS3,Игры,2015-11-01,...,552.0,2689.0,120.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0
2840,133612,48,3708,2015,11,"Gran Turismo 6 [PS3, русская версия]",19,Игры - PS3,Игры,2015-11-01,...,206.0,1152.0,69.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5719,40528,12,5035,2015,11,"NHL 14 [PS3, русские субтитры]",19,Игры - PS3,Игры,2015-11-01,...,132.0,877.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8789,56521,25,2576,2015,11,DEL REY LANA Honeymoon,55,Музыка - CD локального производства,Музыка,2015-11-01,...,1772.0,6884.0,525.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11015,56834,25,12612,2015,11,КНЯZZ Предвестник (фирм.),55,Музыка - CD локального производства,Музыка,2015-11-01,...,1772.0,6884.0,525.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
27890,13295,6,2244,2015,11,CYRUS MILEY Bangerz,55,Музыка - CD локального производства,Музыка,2015-11-01,...,842.0,3178.0,221.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34322,40860,28,3352,2015,11,"FIFA 16 [Xbox 360, русская версия]",23,Игры - XBOX 360,Игры,2015-11-01,...,563.0,2956.0,117.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0
34323,45960,31,3352,2015,11,"FIFA 16 [Xbox 360, русская версия]",23,Игры - XBOX 360,Игры,2015-11-01,...,484.0,2590.0,154.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0


In [21]:
df_test.head()

,ID,shop_id,item_id,year,month,item_name,item_category_id,item_category_name,big_category,date,...,count_aggr_2_quartals_ago_shop_id_item_category_id,count_aggr_year_shop_id_item_category_id,count_aggr_1_month_shop_id_item_category_id,count_aggr_3_months_shop_id_big_category,count_aggr_1_month_ago_shop_id_big_category,count_aggr_6_months_shop_id_big_category,count_aggr_2_quartals_ago_shop_id_big_category,count_aggr_year_shop_id_big_category,count_aggr_1_month_shop_id_big_category,item_cnt_month
0,0,5,5037,2015,11,"NHL 15 [PS3, русские субтитры]",19,Игры - PS3,Игры,2015-11-01,...,181.0,1056.0,75.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,5100,4,5037,2015,11,"NHL 15 [PS3, русские субтитры]",19,Игры - PS3,Игры,2015-11-01,...,134.0,689.0,34.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10200,6,5037,2015,11,"NHL 15 [PS3, русские субтитры]",19,Игры - PS3,Игры,2015-11-01,...,178.0,1041.0,54.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,15300,3,5037,2015,11,"NHL 15 [PS3, русские субтитры]",19,Игры - PS3,Игры,2015-11-01,...,166.0,853.0,45.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,20400,2,5037,2015,11,"NHL 15 [PS3, русские субтитры]",19,Игры - PS3,Игры,2015-11-01,...,197.0,1286.0,70.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [22]:
df_test_manually_made[['ID', 'item_cnt_month']].to_csv('../data/submissions/previous_value_benchmark_man.csv', index=False)